In [18]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [19]:
################################ Setup #####################################

model_name = "model_11"

import pickle
import pandas as pd
import numpy as np
import scipy
import nltk
import matplotlib.pyplot as plt
import re
import pprint
import sklearn.preprocessing as pp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("max_colwidth",500)
pd.set_option("display.max_rows",None)

In [20]:
################################ Read in JSON ##############################
import json

with open('kaggle_train.json') as data_file:
    data_test = json.load(data_file)

##################### Convert json to data frame ###########################
df = pd.io.json.json_normalize(data_test)                       #convert to data frame
df.columns = ["cuisine","ID","org_ing"]                         #rename columns of data frame
df = df.set_index(df["ID"])                                     #set ID as index
del df["ID"]                                                    #delete previous ID
#df["count_ing"] = [len(l) for l in df["org_ing"]]               #add a count ingredients column
#df["count_ing"] = pickle.load(open("count_ing_scaler","rb"))\
#.transform(np.array(df["count_ingredients"]))

In [21]:
############################## Preprocessing functions #####################
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import EnglishStemmer
from nltk.stem.wordnet import WordNetLemmatizer

def clean(l):
    return [[re.sub(r"^\s",r"",re.sub(r"\W"," ",re.sub(r"(.*)(\((.*)oz.(.*)\))(.*)",r"\1\5",i))).lower() for i in r]\
                         for r in l]

def clean_nouns(l):
    return [[re.sub(r"^\s",r"",re.sub(r"\W"," ",re.sub(r"(.*)((.*)oz.(.*)\))(.*)",r"\1\5",i))).lower() for i in r]\
                         for r in l]

def make_ingredients_list(l):
    ingredients = [item for sublist in l for item in sublist]
    
def remove_empty(l):
    if type(l[0]) == list:
        return [list(filter(None,i)) for i in l]
    else:
        return [i.replace(',,',',').replace(', ,',', ') for i in l]

def extract_nouns(text):
    return " ".join([word \
                     for word,tag in nltk.pos_tag(text.split()) \
                     if(tag=="NN" or tag=="NNP" or tag=="NNS" or tag=="NNPS" or tag=="FW")])

def extract_nouns_in_list(l):
    df["noun_ing"] = [extract_nouns(i) for i in l]
    return df["noun_ing"]
    
def re_process(a):
    def re_p(text):
        text = re.sub(r"(.*)\b(chicken|salt)\b(.*)",r"\2",text)                                        #only keep keywords
        text = re.sub(r"(.*)\b(leaves|large|fresh|shredded|\
        plain|crushed|medium|ground)\b(.*)",r"\1\3",text)  #remove common adjectives
        #text = re.sub(r"(.*)\b(cheese|flour|milk|chilies|salt|oil\
        #|chicken|rice|wine|onion|beans|sugar)\b(.*)",r"\2",text)    #only keep keywords
        #text = re.sub("^(water|salt|pepper|oil|butter)$","",text)                               #remove common ingredients
        #text = re.sub(r"(.*)\
        #(ground|low fat|saturated|fresh|medium|flakes|low sodium|juice|dark|black|refried\
        #shredded|grated|extract|pitted|all-purpose|powder|juice|large|green|red|seedless\
        #blanched|sliced|crushed|wedgie|sharp|whole|wholesome|freshly|plain|and)\
        #\s(.*)",r"\1\3",text)                                                                   #remove common adjectives
        text = re.sub(r"(.*)(lime)(.*)",r"(\1)(lemon)(\2)",text)                                #replace synonymes ? (lemmatize)
        return text
    def list_re(b):
        return [re_p(i) for i in b]
    def str_re(b):
        return ",".join([re_p(i) for i in b.split(",")])
    if type(a[0])==list:
        return a.apply(list_re)
    else:
        return a.apply(str_re)
    
def make_str_ing(a):
    return a.apply(",".join)     #string ingredients
    
def make_und_ing(ing_list,ing_str=None):
    if type(ing_str) != pd.core.series.Series:
        ing_str = make_str_ing(ing_list)
    return [i.replace(" ","_").replace(","," ") for i in ing_str]

def make_dtm(a,dtm_name):
    vect = TfidfVectorizer(input="content",strip_accents="ascii")
    vect.fit(list(a))
    pickle.dump(vect,open(model_name+"_"+dtm_name+"_vect.p","wb"))
    return vect.transform(list(a))

def stem(b,stemmer=None):
    def stemm(a):
        if type(a) == list:
            return [stemmer.stem(i) for i in a]
        else:
            return ",".join([stemmer.stem(i) for i in a.split(",")])
    def lemmatizer(a):
        if type(a) == list:
            return [WordNetLemmatizer().lemmatize(i) for i in a]
        else:
            return ",".join([WordNetLemmatizer().lemmatize(i) for i in a.split(",")])
    if stemmer==None:
        return b.apply(lemmatizer)
    else:
        return b.apply(stemm)

In [22]:
########################### Preprocessing ###############################
df["ing1"] = df["org_ing"]
#df["ing1"] = re_process(df["ing1"])
#df["ing1"] = stem(df["ing1"])
#df["ing1"] = clean(df["ing1"])
#df["ing1"] = remove_empty(df["ing1"])
df["ing1"] = make_str_ing(df["ing1"])
dtm1 = make_dtm(df["ing1"],"dtm1")

#df2 = pickle.load(open("df_bckup3.p","rb"))
#df2["ing2"] = df2["nouns_ing"]
#df2["ing2"] = re_process(df2["ing2"])
#df2["ing2"] = stem(df2["ing2"],LancasterStemmer())
#df2["ing2"] = clean_nouns(df2["ing2"])
#df2["ing2"] = remove_empty(df2["ing2"])
#df2["ing2"] = make_und_ing(df2["ing2"])
#dtm2 = make_dtm(df2["ing2"],"dtm2")

#df["ing3"] = df["org_ing"]
#df["ing3"] = re_process(df["ing3"])
#df["ing3"] = stem(df["ing3"],LancasterStemmer())
#df["ing3"] = clean(df["ing3"])
#df["ing3"] = remove_empty(df["ing3"])
#df["ing3"] = make_und_ing(df["ing3"])
#dtm3 = make_dtm(df["ing3"],"dtm3")

dtm = dtm1
#dtm = scipy.sparse.hstack((dtm1,dtm2),format='csr')
print(type(dtm),dtm.shape)

<class 'scipy.sparse.csr.csr_matrix'> (39774, 3005)


In [23]:
###################### For full train data ##################################
x_train = dtm
y_train = np.array(df["cuisine"])

In [24]:
#################### Setting Training and Testing Data #####################
from sklearn import tree
from sklearn import linear_model
from sklearn import cross_validation
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from time import time

def classify(model,name):
    start_time = time()
    model = Pipeline([('feature_selection',LinearSVC(penalty="l1",dual=False)),('classification',model)])
    acc = cross_validation.cross_val_score(model,x_train,y_train,cv=5)
    acc = [round(i,4)*100 for i in acc]
    model.fit(x_train,y_train)
    end_time = round(time()-start_time,0)
    print(name,acc,":",np.mean(acc),end_time)
    return acc,model,end_time

In [25]:
###################### Classifying single-iteration classifiers ####################
clfs = [
        (LinearSVC(loss='l2',penalty='l1',dual=False,tol=1e-3),"SVM1")
        ,(MultinomialNB(),"NB"),
        #,(LinearSVC(loss='l2',penalty='l2',dual=False,tol=1e-3),"SVM2")
        #,(linear_model.LogisticRegression(C=1e5),"Logistic Regression")
        ]

y_train = np.array(df["cuisine"])
y_train_org = np.array(df["cuisine"])

best_acc = 0
best_name = ""
for clf,name in clfs:
    a,c,end_time = classify(clf,name)
    with open("log_classifiers.txt","a") as text:
        print(model_name,name,np.mean(a),end_time,file=text,sep=",")
    pickle.dump(c,open(model_name+"_"+name+".p","wb"))
    if np.mean(a) > best_acc:
        best_acc = np.mean(a)
        best_name = name

print(best_name,best_acc)

SVM1 [78.600000000000009, 79.430000000000007, 78.829999999999998, 78.560000000000002, 79.25] : 78.934 668.0
NB [68.510000000000005, 69.289999999999992, 67.659999999999997, 68.230000000000004, 68.489999999999995] : 68.436 440.0
SVM1 78.934


In [ ]:
###################### Classifying multiple-iteration classifiers #################
clfs = [
        #(tree.DecisionTreeClassifier(),"Decision Tree")
        #,(RandomForestClassifier(n_estimators=500,n_jobs=5),"RF1")
        #,(AdaBoostClassifier((DecisionTreeClassifier(max_depth=3)),n_estimators=500,learning_rate=1),"ADA 1")
        #,(AdaBoostClassifier((DecisionTreeClassifier(max_depth=5)),n_estimators=500,learning_rate=1),"ADA 2")
        #,(AdaBoostClassifier((DecisionTreeClassifier(max_depth=5)),n_estimators=1000,learning_rate=1),"ADA 3")
        #,(AdaBoostClassifier((DecisionTreeClassifier(max_depth=5)),n_estimators=1400,learning_rate=1),"ADA 4")
        (AdaBoostClassifier((DecisionTreeClassifier(max_depth=7)),n_estimators=100,learning_rate=1),"ADA 5")
        #,(AdaBoostClassifier((DecisionTreeClassifier(max_depth=9)),n_estimators=1200,learning_rate=1),"ADA 6")
        ,(RandomForestClassifier(n_estimators=700,n_jobs=5),"RF2")
        ]

n = 8

best_acc = 0
best_name = ""
for clf,name in clfs:
    a = [0] * n
    c = [0] * n
    for i in range(0,n):
        a[i],c[i] = classify(clf,name)
        if a[i] > best_acc:
            best_acc = a[i]
            best_name = name
    pickle.dump(c[a.index(max(a))],open(model_name+"_"+name+".p","wb"))

print(best_name,best_acc)

In [ ]:
######### Percentage errors in individual cuisines reults for cv #########
from collections import Counter
a=Counter(c.predict(x_cv))
b=Counter(y_cv)
d=pd.DataFrame([(i[0],i[1],b[i[0]] - i[1],round((b[i[0]] - i[1])/i[1]*100)) for i in a.items()]).sort_values(by=3,ascending=False)
d.columns = ["cuisine","total","absolute false negatives","percentage false negatives"]
with open("error_cuisines.txt","w") as text:
    print(d,file=text)
d

In [ ]:
################################ Intro #####################################

#clean : (oz.),keep alpha_num,lowercase
#extract nouns
#regex
#individual dtm
#adding of model-2 level-1 prediction feature

#cv : 5-fold